<a href="https://colab.research.google.com/github/gmoraissc/Projetos_de_Data_Science/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALLATIONS

In [128]:
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2 MB 56.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [129]:
!pip install emoji

     |████████████████████████████████| 174 kB 7.6 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=dd01cecc4986b63d1c3b10d12df5a63d7e72af8414ae15325965c9705d8c2915
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [130]:
!pip install xlsxwriter

IMPORTS

In [131]:
pip install googletrans==4.0.0-rc1

In [132]:
# imports
import pandas as pd
import numpy as np
import tweepy, nltk, json, os, csv, xlsxwriter, pytz, re, heapq, string, spacy, glob, pathlib, time, functools
from pandas import ExcelWriter as ExcelWriter
from pathlib import Path
from datetime import datetime, date, timedelta
from csv import writer
from abc import ABCMeta, abstractmethod
from nltk.corpus import treebank
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import PositiveNaiveBayesClassifier
from textblob import TextBlob
from googletrans import Translator
from itertools import combinations
%matplotlib inline

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [133]:
nlp = spacy.load('/usr/local/lib/python3.7/dist-packages/pt_core_news_sm/pt_core_news_sm-2.2.5')

In [134]:
# fazer auth_Twitter uma subclasse de pesquisar_tweets para herdar o api

In [135]:
class auth_twitter(object):
  
  def __init__(self):
    pass

  def parametros_de_acesso(self, arquivo):
    
    self.__arquivo = arquivo

    with open (arquivo, 'r') as arquivo:
      texto = arquivo.readlines()[0].split(',')
    
    access_token = texto[0].split('=')[1]
    access_token_secret = texto[1].split('=')[1]
    bearer_token = texto[2].split('=')[1]
    consumer_key = texto[3].split('=')[1]
    consumer_secret = texto[4].split('=')[1]

    arquivo.close()

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

DATA EXCTRACTION

In [136]:
class pesquisar_tweets(object):

  def __init__(self):
    self.resultados_da_pesquisa = list()
    self.historico_info_participantes = list()
    self.informacoes_hist_participante = list()
    self.perfis_a_excluir = set(termos_de_pesquisa().participantes_totais()) | set(termos_de_pesquisa().perfis_oficiais_bbb)

  @functools.lru_cache(maxsize=32)
  def publico(self, query: tuple, result_type='popular', count=int(10)):

    "Esta consulta por tweets exclui perfis oficiais ou de participantes para torná-la puramente pública."

    global api

    self.__query = query
    self.__result_type = result_type
    self.__count = count

    for status in tweepy.Cursor(api.search,
                                    q=self.__query,
                                    result_type=self.__result_type,
                                    count=self.__count,
                                    exclude_replies=True).items(1000):

            _ = json.dumps(status._json)
            x = json.loads(_)
            if x['user']['screen_name'] not in self.perfis_a_excluir:
              self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type, 'Publico'
 
  @functools.lru_cache(maxsize=128)
  def por_termo(self, query: tuple, result_type='recent', count=int(100)):
    
    """Este método faz a consulta de tweets a partir de termos. 
    Query: a query deve ser uma tupla (str) utilizando-se de operadores lógicos
    tais como OR, AND. Ex. BBB AND #REDEBBB. 
    Result_type: pode ser ou popular ou recent. O primeiro buscará os tweets
    que corresponderem a query publicados recentemente. O segundo será aqueles
    tweets mais populares. O padrão é recent.
    Count: recebe um valor inteiro para limitar o número de tweets coletados por consulta. O padrão é 100.
    Retorna os resultados da pesquisa (lista) e o tipo de resultado (popular ou recent).
    """

    global api

    self.__query = query
    self.__result_type = result_type
    self.__count = count

    for status in tweepy.Cursor(api.search,
                                q=self.__query,
                                result_type=self.__result_type,
                                count=self.__count).items():

                                self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type
  
  @functools.lru_cache(maxsize=128)
  def por_usuario(self, participante: tuple, result_type: str ='recent', count: int =int(100)):
    
    """Este método faz a consulta de tweets a partir do nome de um usuário. 
    Participante: recebe o screen_name do usuário do twitter. 
    Result_type: pode ser ou popular ou recent. O primeiro buscará os tweets
    que corresponderem a query publicados recentemente. O segundo será aqueles
    tweets mais populares. O padrão é recent.
    Count: recebe um valor inteiro para limitar o número de tweets coletados por consulta. O padrão é 100.
    Retorna os resultados da pesquisa em uma lista, o tipo de resultado (popular ou recent) e o tipo de
    participante (camarote, pipoca, público ou perfil oficial).
    """

    global api

    self.__participante = participante
    self.__result_type = result_type
    self.__count = count
    
    tipo_participante = informacoes_de_participantes().checar_tipo_participante(self.__participante)

    for status in tweepy.Cursor(api.user_timeline,
                                screen_name=self.__participante,
                                result_type=self.__result_type,
                                exclude_replies=False).items(self.__count):
                                
                                self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type, tipo_participante

  
  def por_usuarios(self, usuarios: tuple, result_type: str='recent', count: int =int(100)):
    
    """Este método faz a consulta de tweets a partir do nome de pelo menos um usuário. 
    Participante: recebe o screen_name dos usuários do twitter que deseja ser consultado. 
    Result_type: pode ser ou popular ou recent. O primeiro buscará os tweets
    que corresponderem a query publicados recentemente. O segundo será aqueles
    tweets mais populares. O padrão é recent.
    Count: recebe um valor inteiro para limitar o número de tweets coletados por consulta. O padrão é 100.
    Retorna uma lista com os resultados da pesquisa, o tipo de resultado (popular ou recent) e o tipo de pesquisa (única ou múltipla).
    """

    global api

    self.__result_type = result_type
    self.__count = count
    self.__usuarios = usuarios
    
    for self._participante in self.__usuarios:

      for status in tweepy.Cursor(api.user_timeline,
                                  screen_name=self._participante,
                                  result_type=self.__result_type,
                                  exclude_replies=False).items(self.__count):
                                  
                                  self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type, 'Todos'

  @functools.lru_cache(maxsize=128)
  def infos_do_usuario(self, usuario: str) -> list:
  
    """Este método faz a consulta de informações do usuário (número de seguidores (int), número de perfis que segue (int),
    número de listados (int), número de tweets favoritados (int), número de tweets totais (int), verificado (booleano) e a data
    é uma informação gerada pelo método "corrigir_timezone" cujo padrão é a hora, a fim de organizar os arquivos nas devidas pastas). 
    Usuario: recebe o screen_name do usuário do twitter que deseja ser consultado.
    Retorna uma lista.
    """

    global api

    self.usuario = usuario

    self.__user_results = api.get_user(self.usuario)

    extracao = organizar_coletas_de_tweets.corrigir_timezone(tipo='hora')
    
    self.informacoes_hist_participante.append({'participante': str(self.__user_results.name),
                                          'twitter': str(self.__user_results.screen_name),
                                          'n_seguidores': int(self.__user_results.followers_count),
                                          'n_seguindo': int(self.__user_results.friends_count),
                                          'n_de_listados': int(self.__user_results.listed_count),
                                          'n_tweets_favoritados': int(self.__user_results.favourites_count),
                                          'num_de_tweets_totais': int(self.__user_results.statuses_count),
                                          'verificado': bool(self.__user_results.verified),
                                          'data': extracao})
    
    return self.informacoes_hist_participante

  
  def historico_participante(self, participantes: list):

    """Este métood visa, a partir da busa por usuario, iterar em uma lista de usuários e fazer uma varredura por todos os participantes
    buscando as informações históricas (citadas em infos_do_usuario).
    Participantes: recebe uma lista de screen_names para iteração.
    Retorna uma lista com todas as informações coletadas.
    """

    for self._participante in participantes:
      _ = self.infos_do_usuario(self._participante)
      self.historico_info_participantes.extend(_)
    
    return self.historico_info_participantes, 'historico', 'todos'

DATA ORGANIZATION/FILE HANDLING

In [137]:
class organizar_coletas_de_tweets(object):

  def __init__(self):
    self.dicionario = {}
    self.lista = []
    self.arquivo_consolidado = []
    self.arquivos_a_consolidar = []
    self.consolidados = f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/Consolidados/'
    self.diretorio = f"/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/"

  def criar_dicionario(self, tweets_status, tipo):

    d1 = organizar_coletas_de_tweets.corrigir_timezone(tipo='data')

    self.__tweets_status = tweets_status
    self.__tipo = tipo

    for each_json_tweet in tweets_status:
      _ = json.dumps(each_json_tweet._json)
      
      if d1 not in self.dicionario.keys():
        tweet = {d1: [json.loads(_)]}
        self.dicionario.update(tweet)
        
      else:
        self.dicionario[d1].append(json.loads(_))
      
    return self.dicionario, self.__tipo

  def dataframe_tweets(self, dicionario, tipo_de_pesquisa, multipla=False):
    
    d1 = organizar_coletas_de_tweets.corrigir_timezone(tipo='data')
    
    self.__dicionario = dicionario
    self.__tipo_de_pesquisa = tipo_de_pesquisa

    for i in range(0, len(dicionario[d1])):

          tweet_id = self.__dicionario[d1][i]['id']
          text = self.__dicionario[d1][i]['text']
          favorite_count = self.__dicionario[d1][i]['favorite_count']
          retweet_count = self.__dicionario[d1][i]['retweet_count']
          created_at = self.__dicionario[d1][i]['created_at']
          hashtags = self.__dicionario[d1][i]['entities']['hashtags']
          user_mentions = self.__dicionario[d1][i]['entities']['user_mentions']
          name = self.__dicionario[d1][i]['user']['name']
          screen_name = self.__dicionario[d1][i]['user']['screen_name']

          self.lista.append({'tweet_id': str(tweet_id),
                        'text': str(text),
                        'favorite_count': int(favorite_count),
                        'retweet_count': int(retweet_count),
                        'created_at': created_at,
                        'user_mentions': user_mentions,
                        'name': name,
                        'screen_name': screen_name})
          
    self.__tweet_json_ = pd.DataFrame(self.lista, columns = 
                              ['tweet_id', 'text', 
                                'favorite_count', 'retweet_count', 
                                'created_at', 'hashtags', 
                                'user_mentions', 'name',
                                'screen_name'])
    
    if multipla == False:
      
      self.__tipo_participante = informacoes_de_participantes().checar_tipo_participante(screen_name)
    
    else:
      
      self.__tipo_participante = 'Todos'

    return self.__tweet_json_, self.__tipo_participante, self.__tipo_de_pesquisa

  def dataframe_dos_participantes(self, informacoes_hist_participante):

      self.__informacoes_hist_participante = informacoes_hist_participante

      self.__dados_participantes = pd.DataFrame(self.__informacoes_hist_participante, columns = 
                                      ['participante', 'twitter', 'n_seguidores', 
                                      'n_seguindo','n_de_listados', 'n_tweets_favoritados',
                                      'num_de_tweets_totais', 'verificado', 'data'])
      
      return self.__dados_participantes, 'historico', None
  
  @staticmethod
  def salvar_arquivo(dataframe, tipo_pesquisa, tipo_participante):
    
    data_extracao = organizar_coletas_de_tweets.corrigir_timezone(tipo='data')
    dir_csvs = f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/'

    if tipo_pesquisa == 'historico':
           
      file_variable_ = dir_csvs + 'Participantes'
      file_name = 'historico.csv'
      file_path_ = file_variable_ + "/" + file_name
      organizar_coletas_de_tweets.diretorios(file_variable=file_variable_, file_path=file_path_, dataframe=dataframe)
    
    else:

      file_variable_ = dir_csvs + "/" + tipo_participante  + "/" + tipo_pesquisa + "/" + data_extracao
      file_name = tipo_participante[0:3] + "_" + tipo_pesquisa[0:3] + "_" + data_extracao + '.csv'
      file_path_ = file_variable_ + "/" + file_name
      organizar_coletas_de_tweets.diretorios(file_variable=file_variable_, file_path=file_path_, dataframe=dataframe)

  @staticmethod     
  def diretorios(file_variable, file_path, dataframe):

    if not os.path.exists(file_variable):
      os.makedirs(file_variable)
      dataframe.to_csv(file_path, index=False)

    else:
      with open(file_path, 'a') as f_object:
        writer_object = writer(f_object, delimiter=',')
        
        linhas = []
        
        for row in range(0, len(dataframe)):
          linhas.append([row for row in dataframe.iloc[row]])
          writer_object.writerow(linhas[row])

        f_object.close()
    
  @staticmethod
  def corrigir_timezone(tipo):
    
    utcmoment_naive = datetime.utcnow()
    utcmoment = utcmoment_naive.replace(tzinfo=pytz.utc)
    tz = 'America/Sao_Paulo'
    extracao = utcmoment.astimezone(pytz.timezone(tz)) - timedelta(hours=0, minutes=60)
    data_de_extracao = extracao.date().strftime("%d%m%Y")

    if tipo == 'data':
      return data_de_extracao
    
    elif tipo == 'hora':
      return extracao

  
#arquivos_consolidados = {'Camarote':[],
#                         'Pipoca': [],
#                         'Publico': [],
#                         'PerfilOficial': [],
#                         'Todos': []}
  
  def consolidar(self, name, path, tipo):

    self.arquivo_consolidado.append(name)
    self.arquivos_a_consolidar.append(pathlib.PurePath(path, name))
    self.combined_csv = pd.concat([pd.read_csv(f, usecols=['tweet_id', 'text', 'screen_name']) for f in self.arquivos_a_consolidar])
    self.diretorio_a_consolidar = pathlib.PurePath(self.consolidados + tipo,'consolidado_' + tipo + '.csv')
    self.combined_csv.to_csv(self.diretorio_a_consolidar, index=False )

  
  def consolidar_total(self):

    for path, dirs, files in os.walk(self.diretorio, topdown=False):
      
      for name in files:
        if 'Camarote' in path:
          _ = 'Camarote'
          self.consolidar(name=name, path=path, tipo=_)
                
        elif 'Pipoca' in path:
          _ = 'Pipoca'
          self.consolidar(name=name, path=path, tipo=_)

        elif 'Publico' in path:
          _ = 'Publico'
          self.consolidar(name=name, path=path, tipo=_)
        
        elif 'PerfilOficial' in path:
          _ = 'PerfilOficial'
          self.consolidar(name=name, path=path, tipo=_)
        
        elif 'Todos' in path:
          _ = 'Todos'
          self.consolidar(name=name, path=path, tipo=_)


PARTICIPANTS INFORMATIONS

In [138]:
class informacoes_de_participantes(object):

  def __init__(self):
    pass

  @staticmethod
  def checar_tipo_participante(usuario):
      if usuario in termos_de_pesquisa().participantes_camarote:
        return 'Camarote'
      elif usuario in termos_de_pesquisa().participantes_pipoca:
        return 'Pipoca'
      elif usuario in termos_de_pesquisa().perfis_oficiais_bbb:
        return 'PerfilOficial'
      else:
        return 'Publico'

TEXT OPERATIONS

In [139]:
class termos_de_pesquisa(object):
  
  def __init__(self):
    self.__participantes_totais = []
    self.participantes_camarote = ['Aguiarthur', 'Naiarazevedo', 'PedroScooby', 'brunnagoncalves', 
                          'iampauloandre', 'eumaria', 'jadepicon', 'Silva_DG', 'linndaquebrada', 'TiagoAbravanel']

    self.participantes_pipoca = ['Dra_laiscaldass', 'LucianoEstevan', 'a_jessilane', 'Eli', 'eslomarques', 
                      'bbaheck', 'oficialmussi', 'oficial_deodato', 'vyniof', 'LucasBissoli_']

    self.participantes_camarote_twitter = ['@Aguiarthur', '@Naiarazevedo', '@PedroScooby', '@brunnagoncalves', 
                        '@iampauloandre', '@eumaria', '@jadepicon', '@Silva_DG', '@linndaquebrada', '@TiagoAbravanel']

    self.participantes_pipoca_twitter = ['@Dra_laiscaldass', '@LucianoEstevan', '@a_jessilane', '@eusouoeli', '@eslomarques', 
                      '@bbaheck', '@oficialmussi', '@oficial_deodato', '@vyniof', '@LucasBissoli_']

    self.perfis_oficiais_bbb = ['bbb', 'globoplay', 'tadeuschmidt']

    self.perfis_oficiais_bbb_twitter = ['@bbb', '@globoplay', '@tadeuschmidt']

  def termos(self):
    
    self.dicionario_de_termos_de_pesquisa = {'BigBrotherBrasil': {'BBB', 'BBB22', '#REDEBBB', '#BBB', '#BBB22'},
         'Participantes': {'Arthur': ['Arthur', 'Artur', 'Aguiar', 'Arthur Aguiar', '\u2747\ufe0f', '#TeamAguiar', '#TeamArthurAguiar'],
                           'Naiara Azevedo': ['Naiara', 'Azevedo', '1f4b8', '#TeamNaiara', 'Nai', 'cantora'],
                           'Pedro Scooby': ['Pedro', 'Scooby', '1f30a', '#TeamScooby', '#TimeScooby', '1f499'],
                           'Brunna Gonçalves': ['Brunna', 'Gonçalves', '1f984', '#BBBRUNNA', 'BBBrunna'],
                           'Paulo André': ['Paulo', 'André', '1f3c1', '#TeamPauloAndré', 'TeamPauloAndre'],
                           'Maria': ['Maria', '1f40d', '#TEAMMARIA', 'TIME MARIA', 'MARICONAS'],
                           'Jade Picon': ['Jade', 'Picon', '1f32a\ufe0f', 'furacão', '#TeamJade', 'Picão', 'KdJade', 'Picões', 'Furacao'],
                           'Douglas Silva': ['Douglas', 'Silva', '1f3b2', 'dado', 'dadinho', '#TeamDouglasSilva', '1f44a\U0001f3ff'],
                           'Linn da Quebrada': ['Linn', 'Quebrada', '1f9dc\u200d\u2640\ufe0f', '1f9dc\U0001f3ff\u200d\u2640\ufe0f',
                                                '1f9dc\U0001f3fb\u200d\u2640\ufe0f', '1f9dc\U0001f3fe\u200d\u2640\ufe0f', '1f9dc\U0001f3fc\u200d\u2640\ufe0f',
                                                '1f9dc\U0001f3fd\u200d\u2640\ufe0f', '1f9dc', '1f9dc\U0001f3ff', '1f9dc\U0001f3fb', '1f9dc\U0001f3fe',
                                                '1f9dc\U0001f3fc', '1f9dc\U0001f3fd', '#TeamLinn', '#linndonas', '#LinnDonas'],
                           'Tiago Abravanel': ['Tiago', 'Abravanel', '1f43b', '#TeamAbrava', '#TeamAbravanel', '1f9f8'],
                           'Laís Caldas': ['Laís', 'Caldas', '1f462', '#TeamLais', '#TeamLaís', 'Time Lais', '1f49a'],
                           'Luciano Estevan': ['Luciano', 'Estevan', '1f981', '#TeamLucianoEstevan', 'Lu', '1f346'],
                           'Jessilane': ['Jessilane', '1f9ec', 'Jessi', '#TeamJessi', '#TimeJessi', '1f49c', 'Charmanders'],
                           'Eliezer': ['Eliezer', '1F437', 'Eli', '#TeamEli', '1F953', '1F416', '1F43D'],
                           'Eslovênia Marques': ['Eslovênia', 'Eslováquia', 'Marques', 'Eslô', '1f1f8\U0001f1ee',
                                                 'Time Eslo', 'Team Eslo', '#TimeEslô', '#TeamEslô', 'Eslovenia'],
                           'Bábara Heck': ['Bárbara', 'Heck', '1f980', '#TeamBá', 'TeamBá', 'TimeBá',
                                           'TimeBa', 'TeamBa', 'TeamBah', 'BBBah', 'Bá', 'Ba', '1f9a6'],
                           'Rodrigo Mussi': ['Rodrigo Mussi', '#TeamMussi', "BBB", "BIG BROTHER BRASIL",
                                             '#TimeMussi', '1f977\U0001f3ff', '1f977', '1f977\U0001f3fb',
                                             '1f977\U0001f3fe', '1f977\U0001f3fc', '1f977\U0001f3fd', 'ninja', 'ninjas', '#TEAMNINJA'],
                           'Natália Deodato': ['Natália', 'Deodato', '#TeamNaty', '#TimeNaty', 'Naty', 'vitilindos',
                                               '1f483', '1f483\U0001f3ff', '1f483\U0001f3fb',
                                               '1f483\U0001f3fe', '1f483\U0001f3fc', '1f483\U0001f3fd'],
                           'Vinicius': ['Vinicius', 'TeamVyni', '#TimeVyni', '#TeamVyni', '1f4a1'],
                           'Lucas Bissoli': ['Lucas', 'Bissoli', '#TimeBissoli', '#TeamBissoli', 
                                             '1f3c4\u200d\u2642\ufe0f', '1f3c4\U0001f3ff\u200d\u2642\ufe0f', 
                                             '1f3c4\U0001f3fb\u200d\u2642\ufe0f', '1f3c4\U0001f3fe\u200d\u2642\ufe0f',
                                             '1f3c4\U0001f3fc\u200d\u2642\ufe0f', '1f3c4\U0001f3fd\u200d\u2642\ufe0f',
                                             '1f3c4', '1f3c4\U0001f3ff', '1f3c4\U0001f3fb', '1f3c4\U0001f3fe',
                                             '1f3c4\U0001f3fc', '1f3c4\U0001f3fd', '1f3c4\u200d\u2640\ufe0f',
                                             '1f3c4\U0001f3ff\u200d\u2640\ufe0f', '1f3c4\U0001f3fb\u200d\u2640\ufe0f',
                                             '1f3c4\U0001f3fe\u200d\u2640\ufe0f', '1f3c4\U0001f3fc\u200d\u2640\ufe0f',
                                             '1f3c4\U0001f3fd\u200d\u2640\ufe0f']}}
    return self.dicionario_de_termos_de_pesquisa
    #termos_a_analisar = []

    #avaliador_de_termos()

    #return termos_de_pesquisa_BBB

    #analisador_de_termos(termos_de_pesquisa_participantes)

  def analisador_de_termos(self, termos_atuais, termos_novos):

    self.__termos_atuais = termos_atuais
    self.__termos_novos = termos_novos

    #for termo in termos_atuais:
      # if novo termo ajudar a explicar:
      # append
      # if len(termos) > limite:
      # termo_a_remover = ()
      # for termo in termos atuais atualizado:
      # termo_a_remover['explicacao'] < min_a_explicar:
      # termos_atuais.pop(termo_a_remover)

  def tendencia(self, termo_novo):

    self.termo_novo = termo_novo
    pass

  def correlacao_participantes(self):
    pass    

  def dicionario_participantes(self):

    self.dicionario_de_participantes = {'@Aguiarthur': 'Arthur Aguiar', 
                            '@Naiarazevedo': 'Naiara Azevedo',
                            '@PedroScooby': 'Pedro Scooby',
                            '@brunnagoncalves': 'Bruna Gonçalves',
                            '@iampauloandre': 'Paulo André',
                            '@eumaria': 'Maria',
                            '@jadepicon': 'Jade Picon',
                            '@Silva_DG': 'Douglas Silva',
                            '@linndaquebrada': 'Linn da Quebrada',
                            '@TiagoAbravanel': 'Tiago Abravanel',
                            '@Dra_laiscaldass': 'Laís Caldas',
                            '@LucianoEstevan': 'Luciano Estevan',
                            '@a_jessilane': 'Jessilane',
                            '@eusouoeli': 'Eliezer',
                            '@eslomarques': 'Eslovênia Maques',
                            '@bbaheck': 'Bábara Heck',
                            '@oficialmussi': 'Rodrigo Mussi',
                            '@oficial_deodato': 'Natália Deodato',
                            '@vyniof': 'Vinicius',
                            '@LucasBissoli_': 'Lucas Bissoli'}

    return self.dicionario_de_participantes
  
  def lista_participantes(self, tipo_participante, perfil=True):     

    if tipo_participante == 'camarote':
      if perfil:
        return self.participantes_camarote_twitter
      else:
        return self.participantes_camarote
    elif tipo_participante == 'pipoca':
      if perfil:
        return self.participantes_pipoca_twitter
      else:
        return self.participantes_pipoca
    elif tipo_participante == 'oficial':
      if perfil:
        return self.perfis_oficiais_bbb_twitter
      else:
        return self.perfis_oficiais_bbb

  def participantes_totais(self):

    self.__participantes_totais.extend(self.participantes_pipoca)
    self.__participantes_totais.extend(self.participantes_camarote)

    return self.__participantes_totais

TESTES

In [140]:
api = auth_twitter().parametros_de_acesso('/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/auth.txt')
pesquisar = pesquisar_tweets()
organizar = organizar_coletas_de_tweets()
pesquisas = termos_de_pesquisa()

PESQUISAS POR NOME DE USUARIO(S)

In [141]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.por_usuarios(usuarios=pesquisas.participantes_totais())
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa, multipla=True)
organizar.salvar_arquivo(tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe, tipo_participante=tipo_de_participante)

HISTÓRICO DOS PARTICIPANTES

In [142]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.historico_participante(pesquisas.participantes_totais())
dataframe, tipo_de_pesquisa, tipo_de_participante = organizar.dataframe_dos_participantes(tweets)
organizar.salvar_arquivo(dataframe=dataframe, tipo_pesquisa=tipo_de_pesquisa, tipo_participante=tipo_de_participante)

PESQUISA PÚBLICO

In [143]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.publico('BBB -filter:retweets', result_type='popular')
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa, multipla=False)
organizar.salvar_arquivo(tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe, tipo_participante=tipo_de_participante)

PESQUISA POR TERMO

In [144]:
tweets, tipo_de_pesquisa = pesquisar.por_termo(query="(BBB22 -tag -mutirão) -from:bbb -from:globoplay -from:tracklist -from:bchartsnet -from:multishow -is:retweet", result_type='popular', count=100)
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa)
organizar.salvar_arquivo(tipo_participante=tipo_participante, tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe)

PESQUISA PERFIL OFICIAL

In [145]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.por_usuario('@bbb')
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa, multipla=False)
organizar.salvar_arquivo(tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe, tipo_participante=tipo_de_participante)

In [146]:
organizar.consolidar_total()

NLP

In [125]:
class processamento_de_texto(object):

  def __init__(self):
    self.texto_processado = []
    self.texto_sendo_analisado = []
    self.texto_sem_pontuacao = []
    self.texto_sem_stopwords = []
    self.texto_lematizado = []
    self.texto_tokenizado = []
 
  @functools.lru_cache(maxsize=32)
  def remover_https_e_espacamento(self, x): 

      self.__sem_https = re.sub(r'https.+', '', str(x))
      self.__sem_arroba = re.sub(r'@+', '', self.__sem_https)
      self.__sem_caractere_de_espacamento = re.sub(r'\n+', ' ', self.__sem_arroba)

      return self.__sem_caractere_de_espacamento
  
  @functools.lru_cache(maxsize=32)
  def remover_pontuacao(self, x):
    
      self.__s = re.sub(r'\W', ' ', x)
      self.__x = re.sub(' +', ' ', self.__s)
      
      return self.__x

  @functools.lru_cache(maxsize=32)
  def remover_stopwords(self, x):  

      return " ".join([palavra for palavra in x.split() if palavra.lower() not in nltk.corpus.stopwords.words('portuguese')])

  @functools.lru_cache(maxsize=32)
  def lemmatizar(self, x):

      return ' '.join([token.lemma_ for token in nlp(x)])
  
  @functools.lru_cache(maxsize=32)
  def tokenizar_texto(self, x):
        
      return word_tokenize(x)

  @functools.lru_cache(maxsize=32)
  def traduzir(self, x):

    if x is not None and not len(x)==0:
      time.sleep(1)
      return Translator().translate(x, dest='en').text
      
    else:
      time.sleep(1)
      return x

  @functools.lru_cache(maxsize=32)
  def preprocessar(self, x):

    self.__sem_link_nem_espaco = self.remover_https_e_espacamento(x)
    self.__sem_pontuacao = self.remover_pontuacao(self.__sem_link_nem_espaco)  
    self.__sem_stopwords = self.remover_stopwords(self.__sem_pontuacao)
    self.__lemmatizado = self.lemmatizar(self.__sem_stopwords)
    self.__traduzido = self.traduzir(self.__lemmatizado)
        
    return self.__traduzido
  
  @functools.lru_cache(maxsize=32)
  def list_reduction(self, texto_processado):

    return [item for sublist in texto_processado[0] for item in sublist]

MELHORIAS

In [127]:
# Funcionalidades
# property para não precisar "herdar" as informações de tipo de pesquisa e tipo de consulta (GETTER)
# try except clauses
# metodo estático para criar a query a partir de um dicionario dado
# metodo para adicionar termos mais relevantes para o modelo *publico e por participante, exemplo, memes
#termos mais frequentes para determinado participante
# verificar se o arquivo já foi consolidado para não "retrabalho"
# coletas automáticas/schedule!


# Analises
# prob de ser eliminado no paredão de acordo com o atual nível de popularidade
# modelo de classificação baseado na popularidade do participante
# em média, quantos seguidores ganha após um tweet positivo? e quantos perde após um tweet negativo?


#wordcloud
#plt.figure(figsize = (20,20))
#wc = WordCloud(max_words = 1000 , width = 1600 , height = 800,
               #collocations=False).generate(" ".join(data_neg))
#plt.imshow(wc)

______________________

In [ ]:
# def status_do_participante -> eliminado? anjo? lider? acompanhar ao longo do tempo e comparar com o "desempenho" no twitter

In [59]:
class processar_bases_de_treino(object):

  def __init__(self):
    self.train_total = pd.read_csv(f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/BaseDeTreino/base_de_treino.csv')

  @functools.lru_cache(maxsize=32)
  def train_fala_do_jogo(self):
    self.__subset = self.train_total[['texto_traduzido', 'jogo']]
    self._train_is_jogo = [tuple(x) for x in self.__subset.to_numpy()]

    return self._train_is_jogo

  @functools.lru_cache(maxsize=32)
  def train_intencao_de_voto(self):
    self.__subset = self.train_total[['texto_traduzido', 'voto']]
    self._train_intencao = [tuple(x) for x in self.__subset.to_numpy()]
    
    return self._train_intencao

  @functools.lru_cache(maxsize=32)
  def train_spam(self):
    self.__subset = self.train_total[['texto_traduzido', 'spam']]
    self._train_is_spam = [tuple(x) for x in self.__subset.to_numpy()]
    
    return self._train_is_spam

  @functools.lru_cache(maxsize=32)
  def train_sentimento(self):
    self.__subset = self.train_total[['texto_traduzido', 'sentimento']]
    self._train_sentimento = [tuple(x) for x in self.__subset.to_numpy()]
    
    return self._train_sentimento

In [126]:
class analises_de_tweets(processar_bases_de_treino):

  def __init__(self):
    super().__init__(self)
    self.cl_sent = NaiveBayesClassifier(super().train_sentimento)
    self.classifier_jogo = PositiveNaiveBayesClassifier(positive_set=[x for x,_ in list(filter(lambda x: x[1] == 1, super().train_fala_do_jogo())) if type(x) == str], unlabeled_set=[x for x,_ in list(filter(lambda x: x[1] == 0, super().train_fala_do_jogo())) if type(x) == str])
    self.classifier_spam = PositiveNaiveBayesClassifier(positive_set=[x for x,_ in list(filter(lambda x: x[1] == 1, super().train_spam())) if type(x) == str], unlabeled_set=[x for x,_ in list(filter(lambda x: x[1] == 0, super().train_spam())) if type(x) == str])
    self.classifier_intencao = PositiveNaiveBayesClassifier(positive_set=[x for x,_ in list(filter(lambda x: x[1] == 1, super().train_intencao_de_voto())) if type(x) == str], unlabeled_set=[x for x,_ in list(filter(lambda x: x[1] == 0, super().train_intencao_de_voto())) if type(x) == str])

  @functools.lru_cache(maxsize=32)
  def classificar_sentimento_do_tweet(self, x):
    return self.cl_sent.classify(x)
  
  @functools.lru_cache(maxsize=32)
  def classificar_tweet_jogo(self, x):
    return self.classifier_jogo.classify(x)
  
  @functools.lru_cache(maxsize=32)
  def classificar_intencao_de_voto(self, x):
    return self.classifier_itencao.classify(x)

  @functools.lru_cache(maxsize=32)
  def classificar_spam(self, x):
    return self.classifier_spam.classify(x)

  @functools.lru_cache(maxsize=32)
  def analisar_sentimento_do_tweet(self, x):
    self.sent = float(TextBlob(x).sentiment[0])
    return self.sent

  @functools.lru_cache(maxsize=32)
  def analisar_polaridade_do_tweet(self, x):
    self.pol = float(TextBlob(x).sentiment[1])
    return self.pol

In [64]:
# olhar para o text e identificar sobre qual jogador (es) está falando?
publico_06022022 = pd.read_csv(f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/Publico/popular/06022022/Pub_pop_06022022.csv')
publico_06022022.head()

,tweet_id,text,favorite_count,retweet_count,created_at,hashtags,user_mentions,name,screen_name
0,1490375498865721353,RT @acrimewavee: lembrando aqui de quando o po...,0,111,Sun Feb 06 17:23:03 +0000 2022,NaN,"[{'screen_name': 'acrimewavee', 'name': 'sâmi'...",Pedrinho #BBB22,psjds_
1,1490375495288033290,RT @_talithafreitas: A única pessoa que entend...,0,288,Sun Feb 06 17:23:02 +0000 2022,NaN,"[{'screen_name': '_talithafreitas', 'name': 'T...",𝒋𝒐𝒚,wtfxjoy
2,1490375494449176578,RT @romanoandre: Até o @chicobarney eles coloc...,0,16,Sun Feb 06 17:23:02 +0000 2022,NaN,"[{'screen_name': 'romanoandre', 'name': 'André...",エリカDamasceno🎧🅾+,DamascenoErica
3,1490375493190828032,RT @Silva_DG: Tamo viciados nessa amizade! Iss...,0,26,Sun Feb 06 17:23:02 +0000 2022,NaN,"[{'screen_name': 'Silva_DG', 'name': 'Douglas ...",Quezia,quezzymarvila
4,1490375493178298368,@readingpace puta\n\nlaís luciano jessilane el...,0,0,Sun Feb 06 17:23:02 +0000 2022,NaN,"[{'screen_name': 'readingpace', 'name': 'nai☁️...",ana,barrowells


In [ ]:
sheet_id = '15_1pGfHuKuLsqbhXLMQnCP5l7LCTyxYktG2gi0ezt44'
sheet_name = 'consolidado_Publico'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
DfConsolidadoPublico = pd.read_csv(url)